In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import pickle

In [17]:
def load_data(images_path, labels_path):
    images = []
    labels = []
    for filename in os.listdir(images_path):
        img_path = os.path.join(images_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))  
        
        label_file = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_path, label_file)
        
        with open(label_path, 'r') as f:
            label_data = f.readlines()
        label = []
        for line in label_data:
            parts = line.split()
            x_min, y_min, x_max, y_max = map(float, parts[1:])
            if len(label) < 5:
                label.append([x_min, y_min, x_max, y_max])
        
        images.append(img)
        while len(label) < 5:
            label.append([0,0,0,0])
        labels.append(label)
    X = np.array(images)
    Y = np.array(labels, dtype=np.float32) 
    Y = Y.reshape((len(Y), 5 * 4))
    return X,Y 

In [18]:
images_path = 'data/images2'
labels_path = 'data/labels/labels'
X, y = load_data(images_path, labels_path)


In [19]:
print(X[0])
print(y[0])
print(f"Toplam Resim Sayısı: {len(X)}")
print(f"Toplam Label Sayısı: {len(y)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[0.39433593 0.32539013 0.139063   0.192188   0.6656245  0.315625
 0.142188   0.189062   0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
Toplam Resim Sayısı: 16733
Toplam Label Sayısı: 16733


In [20]:
max_faces = 5
  
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(max_faces * 4, activation='sigmoid')  
])

c:\Users\emira\OneDrive\Masaüstü\WORK\Mentor\MentorApp\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 375s 1s/step - accuracy: 0.2744 - loss: 0.2120 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 423s 1s/step - accuracy: 0.2741 - loss: 0.2153 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.2747 - loss: 0.2151 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 403s 1s/step - accuracy: 0.2738 - loss: 0.2153 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 403s 1s/step - accuracy: 0.2757 - loss: 0.2156 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 378s 1s/step - accuracy: 0.2765 - loss: 0.2155 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 370s 1s/step - accuracy: 0.2686 - loss: 0.2158 - val_accuracy: 0.2733 - val_loss: 0.2153
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 379s 1s/step - accuracy: 0.2692 - loss: 0.2151 - val_accu

In [23]:
with open('face_detection_model1.pkl', 'wb') as f:
    pickle.dump(model, f)